In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
#  Import and read the charity_data.csv.
import pandas as pd
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Pre-Processing Data
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
app_count = application_df['APPLICATION_TYPE'].value_counts()

replace_application = list(app_count[app_count < 500].index)

for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

class_count = application_df['CLASSIFICATION'].value_counts()
replace_class = list(class_count[class_count < 1800].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)


C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)


C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt/weights.{epoch:02d}.hdf5"

In [7]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch")

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

NameError: name 'nn' is not defined

# More Neurons, More Layers, More Better?

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4400      
                                                                 
 dense_5 (Dense)             (None, 60)                6060      
                                                                 
 dense_6 (Dense)             (None, 20)                1220      
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 11,701
Trainable params: 11,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5682 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546

804/804 [==============================] - 1s 2ms/step - loss: 0.5406 - accuracy: 0.7357
Epoch 73/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5410 - accuracy: 0.7367
Epoch 74/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7376
Epoch 75/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5424 - accuracy: 0.7380
Epoch 76/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5423 - accuracy: 0.7366
Epoch 77/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5421 - accuracy: 0.7375
Epoch 78/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7364
Epoch 79/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7362
Epoch 80/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accuracy: 0.7378
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 

# More is not always better, maybe we need to remove noise data?

In [10]:
pct_90 = application_df['ASK_AMT'].quantile(q=0.9)
pct_95 = application_df['ASK_AMT'].quantile(q=0.95)
pct_99 = application_df['ASK_AMT'].quantile(q=0.99)

In [11]:
print(pct_90, pct_95, pct_99)

196546.2000000002 863507.2999999981 13106354.259999901


In [12]:
application_df_99 = application_df[application_df['ASK_AMT']<pct_99]
application_df_95 = application_df[application_df['ASK_AMT']<pct_95]
application_df_90 = application_df[application_df['ASK_AMT']<pct_90]

In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df_95.IS_SUCCESSFUL.values
X = application_df_95.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 30)                2430      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
764/764 [==============================] - 2s 2ms/step - loss: 0.5658 - accuracy: 0.7240
Epoch 2/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7342
Epoch 3/100
764/764 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7363
Epoch 4/100
7

764/764 [==============================] - 1s 2ms/step - loss: 0.5279 - accuracy: 0.7473
Epoch 74/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5277 - accuracy: 0.7479
Epoch 75/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5286 - accuracy: 0.7471
Epoch 76/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5276 - accuracy: 0.7475
Epoch 77/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5276 - accuracy: 0.7472
Epoch 78/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5269 - accuracy: 0.7476
Epoch 79/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5276 - accuracy: 0.7474
Epoch 80/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5276 - accuracy: 0.7472
Epoch 81/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5274 - accuracy: 0.7469
Epoch 82/100
764/764 [==============================] - 1s 2ms/step - loss: 0.5272 - accuracy: 

# lets get rid of more data

In [21]:
application_df = pd.read_csv("data/charity_data.csv")
#Pre-Processing Data
application_df.drop(columns=['EIN', 'NAME'], inplace=True)


application_df_2 = application_df.drop(columns=['APPLICATION_TYPE','CLASSIFICATION'])    
    
application_cat = application_df_2.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_2[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

application_df_2 = application_df_2.merge(encode_df,left_index=True, right_index=True)
application_df_2 = application_df_2.drop(application_cat,1)



C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [22]:
# Split our preprocessed data into our features and target arrays
y = application_df_2.IS_SUCCESSFUL.values
X = application_df_2.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt_2/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch")

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])


model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 80)                2320      
                                                                 
 dense_18 (Dense)            (None, 30)                2430      
                                                                 
 dense_19 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,781
Trainable params: 4,781
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
756/804 [===========================>..] - ETA: 0s - loss: 0.6087 - accuracy: 0.6947
Epoch 1: saving model to checkpoints_opt_2\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6099 - accuracy: 0.6930
Epoch 2/50
760/804 [===========================>..] - ETA: 0s - loss: 0.

768/804 [===========================>..] - ETA: 0s - loss: 0.5971 - accuracy: 0.7006
Epoch 31: saving model to checkpoints_opt_2\weights.31.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5965 - accuracy: 0.7014
Epoch 32/50
769/804 [===========================>..] - ETA: 0s - loss: 0.5965 - accuracy: 0.7013
Epoch 32: saving model to checkpoints_opt_2\weights.32.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5968 - accuracy: 0.7013
Epoch 33/50
782/804 [============================>.] - ETA: 0s - loss: 0.5960 - accuracy: 0.7026
Epoch 33: saving model to checkpoints_opt_2\weights.33.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5963 - accuracy: 0.7025
Epoch 34/50
780/804 [============================>.] - ETA: 0s - loss: 0.5967 - accuracy: 0.7015
Epoch 34: saving model to checkpoints_opt_2\weights.34.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5966 - accuracy: 0.7019
Epoch 35/50
780/804 [=======

# actually worse...